In [9]:
import re
import locale
import pandas as pd
import requests
import os

from webLib import textFromURLorFile
from bs4 import BeautifulSoup

# pd.set_option('display.max_rows', 100)

# Configura o locale para o formato de número brasileiro
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [11]:
url = 'https://www.campinas.sp.leg.br/transparencia/gastos-com-funcionarios-por-gabinete/2023/fevereiro2023'
texto = textFromURLorFile(url, "teste.txt")

# print(texto)
# https://regex101.com/

custos = []
n = 1
for resultado in re.finditer(r"(.+)\s*\d\s*R\$\s*(.*)$", texto, re.MULTILINE):
    nome = resultado.group(1)
    salario = resultado.group(2)
    print(n, nome, salario)
    n += 1

1 Arnaldo Salvetti 34.730,15
2 Carlinhos Camelô 37.252,65
3 Carmo Luiz 37.252,65
4 Cecílio 37.252,65
5 Debora Pallermo 37.252,65
6 Edison Ribeiro 37.252,65
7 Eduardo Magoga 37.252,65
8 Fernando Mendes 37.252,65
9 Filipe Marchesi 37.252,65
10 Guida Calixto 37.252,65
11 Gustavo Petta 37.252,65
12 Higor do Campo Gde 37.252,65
13 Jair da Farmácia 37.252,65
14 Jorge Schneider 37.252,65
15 Juscelino Barbarense 37.252,65
16 Luiz Carlos Rossini 37.252,65
17 Luiz Henrique Cirilo 37.252,65
18 Major Jaime 37.252,65
19 Marcelo da Farmácia 37.252,65
20 Marcelo Silva 37.252,65
21 Mariana Conti 37.252,65
22 Marrom Cunha 37.252,65
23 Nelson Hossri 37.252,65
24 Otto Alejandro 33.048,48
25 Paolla Miguel 37.252,65
26 Paulo Búfalo 37.252,65
27 Paulo Gaspar 32.207,64
28 Paulo Haddad 37.252,65
29 Permínio 37.252,65
30 Professor Alberto 37.252,65
31 Rodrigo da Farmadic 37.252,65
32 Rubens Gás 37.252,65
33 Zé Carlos 37.252,65


In [13]:
ano = {}
meses = ("janeiro", "fevereiro", "marco", "abril", "maio", "junho", "julho",
         "agosto", "setembro", "agosto", "outubro", "novembro", "dezembro")
for n in range(1, 6, 1):
    url = f'https://www.campinas.sp.leg.br/transparencia/gastos-com-funcionarios-por-gabinete/2023/{meses[n]}2023'
    texto = textFromURLorFile(url, f"dados-camp2023{n:02d}.txt")

    for resultado in re.finditer(r"(.+)\s*\d\s*R\$\s*(.*)$", texto, re.MULTILINE):
        nome = resultado.group(1)
        if nome not in ano:
            ano[nome] = {}

        valor = locale.atof(resultado.group(2))
        ano[nome][f"mes{n}"] = valor
ano

{'Arnaldo Salvetti': {'mes1': 34730.15,
  'mes2': 37252.65,
  'mes3': 37252.65,
  'mes4': 41772.96},
 'Carlinhos Camelô': {'mes1': 37252.65,
  'mes2': 37252.65,
  'mes3': 37252.65,
  'mes4': 41772.96},
 'Carmo Luiz': {'mes1': 37252.65,
  'mes2': 37252.65,
  'mes3': 37252.65,
  'mes4': 41534.61},
 'Cecílio': {'mes1': 37252.65,
  'mes2': 37252.65,
  'mes3': 37252.65,
  'mes4': 41772.96},
 'Debora Pallermo': {'mes1': 37252.65,
  'mes2': 37252.65,
  'mes3': 36916.32,
  'mes4': 41772.96},
 'Edison Ribeiro': {'mes1': 37252.65,
  'mes2': 37252.65,
  'mes3': 37252.65,
  'mes4': 41772.96},
 'Eduardo Magoga': {'mes1': 37252.65,
  'mes2': 37252.65,
  'mes3': 37252.65,
  'mes4': 41772.96},
 'Fernando Mendes': {'mes1': 37252.65,
  'mes2': 37252.65,
  'mes3': 37252.65,
  'mes4': 41772.96},
 'Filipe Marchesi': {'mes1': 37252.65,
  'mes2': 37252.65,
  'mes3': 37252.65,
  'mes4': 41772.96},
 'Guida Calixto': {'mes1': 37252.65,
  'mes2': 37252.65,
  'mes3': 37252.65,
  'mes4': 41772.96},
 'Gustavo Petta

In [20]:
df = pd.DataFrame.from_dict(ano, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'vereador'}, inplace=True)
df.fillna(0, inplace=True)
df['total'] = df.iloc[:, -4:].sum(axis=1)
df = df.sort_values(by='total', ascending=False)
df = df.reset_index(drop=True)
df.to_csv('dados.csv', index=False)
df

,vereador,mes1,mes2,mes3,mes4,total
0,Zé Carlos,37252.65,37252.65,37252.65,41772.96,153530.91
1,Paulo Haddad,37252.65,37252.65,37252.65,41772.96,153530.91
2,Professor Alberto,37252.65,37252.65,37252.65,41772.96,153530.91
3,Rodrigo da Farmadic,37252.65,37252.65,37252.65,41772.96,153530.91
4,Nelson Hossri,37252.65,37252.65,37252.65,41772.96,153530.91
5,Marrom Cunha,37252.65,37252.65,37252.65,41772.96,153530.91
6,Rubens Gás,37252.65,37252.65,37252.65,41772.96,153530.91
7,Marcelo Silva,37252.65,37252.65,37252.65,41772.96,153530.91
8,Major Jaime,37252.65,37252.65,37252.65,41772.96,153530.91
9,Carlinhos Camelô,37252.65,37252.65,37252.65,41772.96,153530.91
